一个不解之谜
耳听为虚，眼见为实。我们不妨先来看一个例子，让你感受下 GIL 为什么会让人不明所以。

比如下面这段很简单的 cpu-bound 代码：



In [102]:
def CountDown(n):
    while n > 0:
        n -= 1
%timeit CountDown(100000000)

6.3 s ± 56.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
from threading import Thread
from time import time

n = 100000000

t1 = Thread(target=CountDown, args=[n // 2])
t2 = Thread(target=CountDown, args=[n // 2])
time1=time()
t1.start()
t2.start()
t1.join()
t2.join()
time2=time()
time=time2-time1
time

10.586245059967041

### GIL 是如何工作的？
下面这张图，就是一个 GIL 在 Python 程序的工作示例。其中，Thread 1、2、3 轮流执行，每一个线程在开始执行时，都会锁住 GIL，以阻止别的线程执行；同样的，每一个线程执行完一段后，会释放 GIL，以允许别的线程开始利用资源。


![avator](https://static001.geekbang.org/resource/image/db/8d/dba8e4a107829d0b72ea513be34fe18d.png)
细心的你可能会发现一个问题：为什么 Python 线程会去主动释放 GIL 呢？毕竟，如果仅仅是要求 Python 线程在开始执行时锁住 GIL，而永远不去释放 GIL，那别的线程就都没有了运行的机会。

没错，CPython 中还有另一个机制，叫做 check_interval，意思是 CPython 解释器会去轮询检查线程 GIL 的锁住情况。每隔一段时间，Python 解释器就会强制当前线程去释放 GIL，这样别的线程才能有执行的机会。

In [98]:
import threading

n = 0

def foo():
    global n
    n += 1

threads = []
for i in range(100):
    t = threading.Thread(target=foo)
    threads.append(t)

for t in threads:
    t.start()

for t in threads:
    t.join()

print(n)


100
